In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
"""
%%capture
!pip install 'grpcio<1.38.0,>=1.22.0'
!pip install farm-haystack
"""
#%%capture
!echo 'pymilvus==1.1.2' > /tmp/constraints.txt
!pip install -c /tmp/constraints.txt farm-haystack

Refer to [this link](https://stackoverflow.com/questions/58201636/contextual-version-conflict-error-google-colab) if there are any issues with ContextualVersionConflict errors in following code block. Basically just restart the runtime and then run the cell below

In [16]:
!MILVUS2_ENABLED=False
import time
from haystack.nodes import FARMReader
# using path from recently uploaded model
model_name_or_path = 'comacrae/roberta-unaugv2'
trained_reader = FARMReader(model_name_or_path=model_name_or_path, use_gpu=True)
t_start = time.time()
eval = trained_reader.eval_on_file('/content/drive/My Drive/hitm/training-datasets/', "newtestdata.json")
delta_t = time.time() - t_start
print(eval)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find comacrae/roberta-unaugv2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded comacrae/roberta-unaugv2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0 
INFO - haystack.modeling.infer -  /w\  /w\
INFO - haystack.modeling.infer -  /'\  / \
INFO - haystack.modeling.data_handler

{'EM': 0.4052863436123348, 'f1': 0.6364856516451793, 'top_n_accuracy': 0.8590308370044053}


In [ ]:
print(eval, delta_t)

{'EM': 0.34801762114537443, 'f1': 0.6231785312168537, 'top_n_accuracy': 0.8634361233480177} 17.484262704849243


Save results

In [17]:
import json, os

# I just wanted a way to compare so I added the EM, F1, and top_n scores and divided it by the time to compute
def get_total_score(scores_dict):
  return (scores_dict['EM'] + scores_dict['f1'] + scores_dict['top_n_accuracy'] ) / scores_dict['seconds_to_compute']

dir_path = '/content/drive/My Drive/hitm/training-datasets/'
filename = 'finetuned-evaluations.json'
path = os.path.join(dir_path,filename)
data = {'data':[]}

if os.path.exists(path):
  with open(path, "r") as f:
    data = json.load(f)
    f.close()
    
eval['model_name'] = model_name_or_path
eval['seconds_to_compute'] = delta_t
eval['total_score'] = get_total_score(eval)
data['data'].append(eval)

print(data)

with open(path, "w+") as f:
  json.dump(data, f)
  f.close()

{'data': [{'EM': 0.14977973568281938, 'f1': 0.3700728950115897, 'top_n_accuracy': 0.8281938325991189, 'model_name': 'deepset/roberta-base-squad2', 'seconds_to_compute': 18.30980110168457, 'total_score': 0.07362430950544312}, {'EM': 0.3568281938325991, 'f1': 0.6256885129902802, 'top_n_accuracy': 0.8810572687224669, 'model_name': 'comacrae/roberta-paraphrasev2', 'seconds_to_compute': 17.304500579833984, 'total_score': 0.10769302280339055}, {'EM': 0.3700440528634361, 'f1': 0.6393522141228677, 'top_n_accuracy': 0.8414096916299559, 'model_name': 'comacrae/roberta-edav2.0', 'seconds_to_compute': 17.494272470474243, 'total_score': 0.10579496585182016}, {'EM': 0.34801762114537443, 'f1': 0.6231785312168537, 'top_n_accuracy': 0.8634361233480177, 'model_name': 'comacrae/roberta-edaparav2', 'seconds_to_compute': 17.484262704849243, 'total_score': 0.10493049130412645}, {'EM': 0.4052863436123348, 'f1': 0.6364856516451793, 'top_n_accuracy': 0.8590308370044053, 'model_name': 'comacrae/roberta-unaugv2'